# Transient diffusion equation

### PDE and boundary conditions
The transient diffusion equation reads

 $$\alpha\frac{\partial c}{\partial t}+\nabla.\left(-D\nabla c\right)=0,$$

 where $c$ is the independent variable (concentration, temperature, etc)
 , $D$ is the diffusion coefficient, and $\alpha$ is a constant.
 
 
 This is an example program for the python extension of the MATLAB toolbox FVTool
 
 Written by Ali A. Eftekhari
 Last checked: June 2021
 
 Ported to python by Gavin M. Weir June, 2023


In [ ]:
import numpy as np
from pyfvtool import *

In [ ]:
# Define the domain and create a mesh structure
L = float(10)   # domain length
Nx = int(20)    # number of cells
Ntetta = int(21)    # 

# Generate the mesh 
m = createMeshRadial2D(Nx=Nx, Ny=Ntetta, Lx=L, Ly=2.0*np.pi)   # Nx: int, Ny: int, Lx: float, Ly: float (Nr, Ntetta, Lr, Tetta)

# Create the boundary condition structure
BC = createBC(m)  # all Neumann boundary condition structure

# Add a Ghost cell
m.cellcenters.x = m.cellcenters.x+1
m.facecenters.x = m.facecenters.x+1

# left boundary
BC.left.a[:3:20] = 0.0    # BC.left.a(1:3:20) = 0
BC.left.b[:3:20] = 1.0    # BC.left.b(1:3:20)=1
BC.lfet.c[:3:20] = 10.0   # BC.left.c(1:3:20)=10 

# right boundary
BC.right.a[:] = 0.0   # BC.right.a(:) = 0
BC.right.b[:] = 1.0   # BC.right.b(:)=1
BC.right.c[:] = 0.0
#
# BC.right.a(1:floor(Ntetta/2)) = 0
# BC.right.b(1:floor(Ntetta/2))=1
# BC.right.c(1:floor(Ntetta/2))=3 

# Top boundary
BC.top.periodic=1;
# BC.top.a[:] = 0.0
# BC.top.b[:] = 1.0 
# BC.top.c[:] = 0.0 

# Bottom boundary
# BC.bottom.a(:) = 0; BC.bottom.b(:)=1; BC.bottom.c(:)=0; % bottom boundary
%% define the transfer coeffs
D_val = 0.01;
D = createCellVariable(m, D_val);
alfa = createCellVariable(m, 1);
u_val=1;
u = createFaceVariable(m, [u_val, 0]);
%% define initial values
c_init = 0.1;
c_old = createCellVariable(m, c_init,BC); % initial values
c = c_old; % assign the old value of the cells to the current values
%% loop
dt = 0.5; % time step
final_t = 7;
Dave = harmonicMean(D);
Mdiff = diffusionTerm(Dave);
[Mbc, RHSbc] = boundaryCondition(BC);
FL = fluxLimiter('Superbee');
Mconv = convectionTerm(u);
for t=dt:dt:final_t
    [M_trans, RHS_trans] = transientTerm(c, dt, alfa);
    M = M_trans+Mconv-Mdiff+Mbc;
    RHS = RHS_trans+RHSbc;
    c = solvePDE(m,M, RHS);
    c_old = c.value;
    figure(1);visualizeCells(c);shading interp; drawnow;
end